In [9]:
# importing Required Packages 
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sn


In [10]:
# Load the STL10 dataset (torchvision)
from torchvision.datasets import STL10
from torchvision import transforms

In [11]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((96, 96)),  # STL10 images are 32x32 -> further the image Dimensions get samller further the model training process run smoothly 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

In [ ]:
#Load the dataset 
stl10 = STL10(root='../datasets', split='train', download=True, transform=transform)

# Flatten the images and get labels
X = torch.stack([x[0].view(-1) for x in stl10])  # Flatten the images
y = torch.tensor([x[1] for x in stl10])  # Get the labels

  1%|          | 18.5M/2.64G [00:49<1:21:04, 539kB/s]

In [2]:
# Convert to numpy
X = X.numpy()
y = y.numpy()

NameError: name 'X' is not defined

In [ ]:
#Train-test split
X_train, X_test, y_train, y_test = \
                        train_test_split(
                            X, y, 
                            test_size=0.2, 
                            random_state=42
                        )

In [ ]:
# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Initialize Models
knn = KNeighborsClassifier(n_neighbors=5)
log_reg = LogisticRegression(max_iter=1000)